## Loading the necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model ## type: ignore
from tensorflow.keras.layers import (Input, Dense, Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling2D, Dropout, concatenate) ## type: ignore 
from tensorflow.keras.applications import ResNet50 ## type: ignore
from sklearn.model_selection import train_test_split ## type: ignore
from sklearn.preprocessing import MinMaxScaler ## type: ignore
from sklearn.metrics import log_loss ## type: ignore
import matplotlib as plt
import math
from functools import partial ## type: ignore
from tqdm import tqdm

## Loading the dataset

In [ ]:
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')
images_path = 'composite_images.npz'
images = np.load(images_path)


In [ ]:
train_data.head()

### Preprocessing the data

In [ ]:
scaler = MinMaxScaler()
train_data['precipitation'] = scaler.fit_transform(train_data[['precipitation']])
test_data['precipitation'] = scaler.transform(test_data[['precipitation']])

In [ ]:

train_data['event_id'] = train_data['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
train_data['event_idx'] = train_data.groupby('event_id', sort=False).ngroup()
test_data['event_id'] = test_data['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
test_data['event_idx'] = test_data.groupby('event_id', sort=False).ngroup()

train_data['event_t'] = train_data.groupby('event_id').cumcount()
test_data['event_t'] = test_data.groupby('event_id').cumcount()

print(train_data.head())
print(test_data.head())

### Decoding the image metadata

In [ ]:
BAND_NAMES = ('B2', 'B3', 'B4', 'B8', 'B11', 'slope')
H, W, NUM_CHANNELS = IMG_DIM = (128, 128, len(BAND_NAMES))
_MAX_INT = np.iinfo(np.int16).max


In [ ]:
def decode_slope(X: np.ndarray) -> np.ndarray:
    return (X / _MAX_INT * (math.pi / 2.0)).astype(np.float32)

def normalize(x: np.ndarray, mean: int, std: int) -> np.ndarray:
    return (x - mean) / std

rough_S2_normalize = partial(normalize, mean=1250, std=500)


In [ ]:
def preprocess_image(x: np.ndarray) -> np.ndarray:
    return np.concatenate([
        rough_S2_normalize(x[..., :-1].astype(np.float32)),
        decode_slope(x[..., -1:]),
    ], axis=-1, dtype=np.float32)

images_path = 'composite_images.npz'
composite_images = np.load(images_path)

In [ ]:
def preprocess_data_and_images(data_df, composite_images):
    event_ids = data_df['event_id'].unique()
    timeseries = []
    labels = []
    images = []

    for event_id in tqdm(event_ids, desc="Processing data"):
        event_data = data_df[data_df['event_id'] == event_id]
        timeseries.append(event_data['precipitation'].values)
        if 'label' in event_data.columns:
            labels.append(event_data['label'].values)
        images.append(preprocess_image(composite_images[event_id]))

    timeseries = np.array(timeseries)
    labels = np.array(labels) if labels else None
    images = np.stack(images, axis=0)

    return timeseries, labels, images


In [ ]:
train_timeseries, train_labels, train_images = preprocess_data_and_images(train_data, composite_images)
test_timeseries, _, test_images = preprocess_data_and_images(test_data, composite_images)


In [ ]:
test_timeseries.shape

### Splitting the data into training and validation splits

In [ ]:
# Train-validation split
train_split, val_split = train_test_split(
    np.arange(len(train_timeseries)), test_size=0.2, random_state=42
)

X_precip_train, X_precip_val = train_timeseries[train_split], train_timeseries[val_split]
y_train, y_val = train_labels[train_split], train_labels[val_split]
X_img_train, X_img_val = train_images[train_split], train_images[val_split]


## Building our model

### Defining our model for processing the timeseries data

In [ ]:
from tensorflow.keras.layers import Lambda #type: ignore    

# Define precipitation model (1D ResNet)
def create_time_series_resnet(input_shape):
    inputs = Input(shape=input_shape)
    reshaped_inputs = Lambda(lambda x: tf.expand_dims(x, axis=-1))(inputs)  # Add an extra dimension
    x = Conv1D(64, kernel_size=3, activation="relu", padding="same")(reshaped_inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(128, kernel_size=3, activation="relu", padding="same")(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(64, activation="relu")(x)
    return Model(inputs, outputs)

precipitation_model = create_time_series_resnet((730,))


### Defining our model for processing the image data

In [ ]:
# Define image model (ResNet)
image_input = Input(shape=(128, 128, 6))
base_model = ResNet50(weights=None, include_top=False, input_tensor=image_input)
image_features = GlobalAveragePooling2D()(base_model.output)
image_model = Model(inputs=image_input, outputs=image_features)


### combining our models


In [ ]:
# Combine models
combined_precip_input = Input(shape=(730,))
combined_precip_features = precipitation_model(combined_precip_input)

combined_image_input = Input(shape=(128, 128, 6))
combined_image_features = image_model(combined_image_input)

combined = concatenate([combined_precip_features, combined_image_features])
x = Dense(128, activation="relu")(combined)
x = Dropout(0.3)(x)
output = Dense(730, activation="sigmoid")(x)
output = Lambda(lambda x: tf.expand_dims(x, axis=-1))(output)  # Reshape to (None, 730, 1)

model = Model(inputs=[combined_precip_input, combined_image_input], outputs=output)


In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

### Defining our callbacks for model optimization

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping #type: ignore

# Create callbacks
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # Reduce learning rate when validation loss plateaus
    factor=0.5,          # Reduce learning rate by a factor of 0.5
    patience=5,          # Wait for 5 epochs before reducing the learning rate
    min_lr=1e-6,         # Minimum learning rate
    verbose=1
)

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Stop training when validation loss stops improving
    patience=10,         # Wait for 10 epochs before stopping
    restore_best_weights=True,  # Restore the best model weights
    verbose=1
)

## Training our model

In [ ]:
# Reshape train_labels and val_labels to match the model's output shape
y_train_reshaped = train_labels[train_split].reshape(-1, 730, 1)
y_val_reshaped = train_labels[val_split].reshape(-1, 730, 1)

# Ensure the model is compiled before training
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train model
history = model.fit(
    [X_precip_train, X_img_train],
    y_train_reshaped,
    validation_data=([X_precip_val, X_img_val], y_val_reshaped),
    epochs=50,
    batch_size=32,
    callbacks=[lr_scheduler, early_stopping],
)


### Evaluating our model on the validation dataset

In [ ]:
y_val_pred = model.predict([X_precip_val, X_img_val])
print(f'Log loss: {log_loss(y_val_reshaped, y_val_pred)}')


### Evaluating model performance using the training plot

In [ ]:
# Plot training history
from matplotlib import pyplot as plt    
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Making predictions on the test set

In [ ]:
y_test_pred = model.predict([test_timeseries, test_images]) 

In [ ]:
y_test_pred.shape

###  Creating a submission file

In [ ]:
sub = pd.read_csv('SampleSubmission.csv')
sub.shape

In [ ]:
submission = pd.DataFrame({
    'event_id': sub['event_id'],
    'label': y_test_pred.flatten()
})

In [ ]:
submission.to_csv("submission_final.csv", index=False)
